In [0]:
df1 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/talati.ak@northeastern.edu/Dallas_profiling.csv")

In [0]:
print("Schema:")
df1.printSchema()

print("Total Rows:", df1.count())
print("Total Columns:", len(df1.columns))

df1.show(5)


Schema:
root
 |-- Restaurant Name: string (nullable = true)
 |-- Inspection Type: string (nullable = true)
 |-- Inspection Date: string (nullable = true)
 |-- Inspection Score: string (nullable = true)
 |-- Street Number: string (nullable = true)
 |-- Street Name: string (nullable = true)
 |-- Street Direction: string (nullable = true)
 |-- Street Type: string (nullable = true)
 |-- Street Unit: string (nullable = true)
 |-- Street Address: string (nullable = true)
 |-- Zip Code: string (nullable = true)
 |-- Violation Description - 1: string (nullable = true)
 |-- Violation Points - 1: string (nullable = true)
 |-- Violation Detail - 1: string (nullable = true)
 |-- Violation Memo - 1: string (nullable = true)
 |-- Violation Description - 2: string (nullable = true)
 |-- Violation Points - 2: string (nullable = true)
 |-- Violation Detail - 2: string (nullable = true)
 |-- Violation Memo - 2: string (nullable = true)
 |-- Violation Description - 3: string (nullable = true)
 |-- Violat

In [0]:
from pyspark.sql.functions import col, sum as _sum, when

print("Missing Value Count:")
null_counts = df1.select([
    _sum(when(col(c).isNull() | (col(c) == ""), 1).otherwise(0)).alias(c) for c in df1.columns
])
null_counts.show(truncate=False)


Missing Value Count:
+---------------+---------------+---------------+----------------+-------------+-----------+----------------+-----------+-----------+--------------+--------+-------------------------+--------------------+--------------------+------------------+-------------------------+--------------------+--------------------+------------------+-------------------------+--------------------+--------------------+------------------+-------------------------+--------------------+--------------------+------------------+-------------------------+--------------------+--------------------+------------------+-------------------------+--------------------+--------------------+------------------+-------------------------+--------------------+--------------------+------------------+-------------------------+--------------------+--------------------+------------------+-------------------------+--------------------+--------------------+------------------+--------------------------+------------

In [0]:
print("Cardinality (Distinct values per column):")
for column in df1.columns:
    print(f"{column}: {df1.select(column).distinct().count()}")


Cardinality (Distinct values per column):
Restaurant Name: 17904
Inspection Type: 4
Inspection Date: 989
Inspection Score: 53
Street Number: 3219
Street Name: 789
Street Direction: 5
Street Type: 20
Street Unit: 828
Street Address: 6693
Zip Code: 147
Violation Description - 1: 461
Violation Points - 1: 4
Violation Detail - 1: 413
Violation Memo - 1: 17915
Violation Description - 2: 472
Violation Points - 2: 9
Violation Detail - 2: 419
Violation Memo - 2: 15884
Violation Description - 3: 473
Violation Points - 3: 12
Violation Detail - 3: 420
Violation Memo - 3: 13497
Violation Description - 4: 451
Violation Points - 4: 13
Violation Detail - 4: 402
Violation Memo - 4: 11432
Violation Description - 5: 415
Violation Points - 5: 14
Violation Detail - 5: 368
Violation Memo - 5: 9492
Violation Description - 6: 401
Violation Points - 6: 11
Violation Detail - 6: 355
Violation Memo - 6: 7657
Violation Description - 7: 383
Violation Points - 7: 11
Violation Detail - 7: 343
Violation Memo - 7: 616

In [0]:
numeric_cols = [f.name for f in df1.schema.fields if str(f.dataType) in ['IntegerType', 'DoubleType', 'LongType', 'FloatType']]
if numeric_cols:
    df1.select(numeric_cols).describe().show()
else:
    print("No numeric columns found.")


No numeric columns found.


In [0]:
cat_cols = [f.name for f in df1.schema.fields if f.dataType.simpleString() == 'string']
print("Top Categories for Categorical Columns:")
for col_name in cat_cols:
    print(f"\n{col_name}:")
    df1.groupBy(col_name).count().orderBy("count", ascending=False).show(5, truncate=False)


Top Categories for Categorical Columns:

Restaurant Name:
+----------------------+-----+
|Restaurant Name       |count|
+----------------------+-----+
|MCDONALDS             |474  |
|BURGER KING           |259  |
|WINGSTOP              |248  |
|SONIC DRIVE IN        |243  |
|CHIPOTLE MEXICAN GRILL|228  |
+----------------------+-----+
only showing top 5 rows


Inspection Type:
+---------------+-----+
|Inspection Type|count|
+---------------+-----+
|Routine        |65765|
|null           |51901|
|Follow-up      |534  |
|Complaint      |12   |
+---------------+-----+


Inspection Date:
+---------------+-----+
|Inspection Date|count|
+---------------+-----+
|null           |51901|
|2022-10-27     |216  |
|2022-11-10     |204  |
|2022-12-21     |195  |
|2024-01-12     |195  |
+---------------+-----+
only showing top 5 rows


Inspection Score:
+----------------+-----+
|Inspection Score|count|
+----------------+-----+
|null            |51901|
|100             |6221 |
|95              |4600 |

In [0]:
total_rows = df1.count()
duplicate_rows = df1.groupBy(df1.columns).count().filter("count > 1").agg({"count": "sum"}).collect()[0][0]
duplicate_rows = duplicate_rows if duplicate_rows else 0
unique_rows = total_rows - duplicate_rows

print(f"Total Rows: {total_rows}")
print(f"Duplicate Rows: {duplicate_rows}")
print(f"Unique Rows: {unique_rows}")


Total Rows: 118212
Duplicate Rows: 118195
Unique Rows: 17
